## 추천시스템 
- 교육세션 복습
- 내용 기반 필터링
    - 콘텐츠의 유사도를 기반으로 사용자에게 추천
    - 특정 유저가 SF와 액션 장르를 좋아했으면 이 유저에게 다른 SF영화 추천
- 협업 필터링
    - 서비스 내에 있느 다수의 사용자로부터 얻은 선호도 정보를 기반으로 사용자에게 추천
    - 비슷한 관심사의 두 사용자에게 한 사용자의 데이터를 바탕으로 다른 사용자에게 추천
    - 사용자 기반: 특정 사용자와 유사한 취향의 사용자가 선호한 아이템 추천
    - 아이템 기반: 사용자가 선호한 아이템과 유사한 아이템 추천
- 하이브리드 필터링
    - 2가지 이상의 다양한 종류의 추천 시스템 알고리즘을 조합하여 만듦

- 협업 필터링 구현 방식: 
- 기억 기반(실제 평점 데이터를 메모리에 저장 후 유사도를 계산하는 방식)
    - 사용자 기반: 나와 비슷한 취향의 사용자를 찾는다. (사용자간 유사도 계산, 이웃들이 선호했지만 내가 아직 보지 않은 아이템 추천) 
    - 아이템 기반: 함께 선호되는 아이템을 찾는다. (모든 아이템 쌍 간의 유사도 계산 후 선호한 아이템과 유사한 아이템 추천
한계: 데이터 희소성 문제(일부만 평가가 있다.) 확장성 문제, 새로운 아이템 문제

-> 모델 기반 협업 필터링의 등장: 내적 연산으로 평점 예측

## 모델 기반
### 잠재 요인, 행렬 분해
잠재요인: 사용자-아이템 상호작용 행렬에 숨겨져 있는, 사용자의 취향이나 아이템의 특성을 설명하는 숨겨진 차원(Hidden Dimension)
행렬 R을 유저 잠재 요인 행렬 P와 아이템 잠재 요인 행렬 Q^T로 분해
P: 각 사용자의 2차원 잠재 요인 벡터
Q: 각 아이템의 2차원 잠재 요인 벡터

평점 예측: 두 잠재 요인의 내적으로 예측 -> 사후 해석

행렬 분해(MF)  
가설: R에 녹아 있는 평점은 결국 사용자의 잠재요인 벡터와 아이템 잠재요인 벡터 내적의 결과이다. 
- 관측된 평점을 가장 잘 재현하는 가장 적절한 잠재요인의 개수 k를 찾고 P와 Q_T를 학습 
=> 평점 R을 예측

방법: SVD(수학적 분해 기법), SGD(최적화 알고리즘), NMF(제약 조건 기반 모델), PMF(확률적 모델)

- SVD (Orthogonality factorization)
- SVD: 어떤 직사각형 행렬이든 세 행렬로 분해할 수 있다.
mxn = mxm * mxn * nxn
Full SVD: 완전 특이값. 모든 특잇값 & 벡터 사용
Truncated SVD: 절단 특이값. 상위 k개의 큰 특이값과 해당 벡터만 사용
-> 결측치 처리 후 -> SVD로 세 벡터 계산 -> 차원 추출 (가장 큰 k개의 특잇값 선택) -> 잠재 요인 추출
- SGD: 목적 함수. 관측된 평점과 예측 평점 사이의 오차 최소화 목적 -> 오차 제곱화 최소화 & 과적합 방지 정규화 수행
- NMF: 분해된 두 개의 행렬의 모든 원소가 0이상 이어야 한다는 제약 조건 추가
    - 구매횟수, 평점 등은 음수일 수 없다는 점 이용
    -> 먼저 p와 q_t를 비음수의 무작위로 초기화 후 최적화(비음수 제약 조건 유지하면서 최소화)
- PMF: 확률적 모델로 접근

## 최근 방향성
### VAE 기반
- 아이템 상호작용 벡터(예: 구매/시청 이력)를 입력 -> 잠재공건에서 재구성 -> 디코더에서 아이템 분포를 복원
- 확률적 잠재 분포로 사용자 선호를 모델링, 희소 데이터에도 강건

- 딥러닝 기반: 비선형성, 확률분포 이용
- AE: 데이터가 부족하면 이 단일 추정치가 불안정, 노이즈에 취약

- VAE의 우수성: 목적함수!
    - 목적함수: 재구성 오차 최소화 목표(원래 입력 x를 최대하게 비슷하게 하려고 함) + KL Divergence 목표 (인코더 출력 잠재 분포가 표준정규분포와 유사하게 강제하여 데이터 과적합과 불규칙 방지.)

=> 복잡하고 불확실성이 반영된 사용자-아이템 패턴을 학습하여 희소 데이터에도 강한 모델 

### GNN 기반
- VAE같은 딥러닝 모델도 관계나 순서 같은 정보 활용 X
- GNN: 사용자와 아이템이 상호작용을 서로 연결된 거대한 관계망으로 표현
- 그래프: Node + Edge
- Node: 현실 세계의 개체
- Edge: 개체 간 관계나 상호작용
-> 비유클리드 공간도 표현 가능

* GNN 작동 구조: 그래프 구조 생성 -> 메시지 패싱(정보 전달) -> 생성, 집계, 갱신(은닉 상태 업데이트) -> 최종 출력
- 핵심 아이디어: 내 이웃 노드의 정보를 나에게 전달/요약
-> 이웃들의 취향을 반영한 최종 임베딩을 얻는다.

- LightGCN: GNN 한 종류
    - GNN에서 특징 변환, 비선형 활성화 함수 제거
    -> 정규화된 이웃 임베딩의 합으로만 구성함. -> 그래프 구조와 전파만 남김

- 활용 사례: Pinterest, 소셜 네트워크, 추천 시스템, 화학 및 생명 과학, 금융 및 보안, traffic, 프로그램 분석
- 장점: 고차원 협업 시그널, 정형화되지 않은 데이터 학습 가능, Cold start 문제 완화에 효과적 (데이터 부족해도 즉시 임베딩 생성)
- 단점: 과평탄화, 과압축, 계산 복잡함, 해석이 어렵다 (순서 반영을 못한다. 신발 -> 바지 -> 셔츠 구매랑 셔츠 -> 바지 -> 신발 순서 구매를 똑같다고 해석)

### Transformer 기반
- CF와 GNN과 같은 모델들은 순차적 정보를 다루는 데 명확한 한계
- 사용자 행동을 순서로 접근 -> 의도 파악. 
- Self Attention   
    -  "이 아이템을 클릭한 것은 30분 전 클릭이 아니라 3일 전 클릭과 더 관련이 있을 수 있다"는 직관(예: 프린터와 토너)을 모델에 반영할 수 있게 해줍니다. (장기 의존성 문제 해결) ㅇㅎㅇㅎ !!
    - 관련성 계산 후 가중치 만들고 이를 기반으로 업데이트함 

- Transformer 기반 주요 추천 시스템 모델: SASRec, BERT4REC

### LLM 기반 
- 추천을 랭킹 문제가 아닌 언어 문제로 (점수 계산이 아닌 문장으로 출력)
- 사용자 상황에 맞게 서술형 추천까지! 
- 대화형 추천과 Zero-shot 추천(학습 데이터에 없던 새로운 아이템도 추천) 할 수 있게 되었다.
- P5: 모든 추천을 LLM이 풀도록 통일(거대)
- TallRec: 범용 LLM을 튜닝하는 프레임 워크

### 현업
- 모델 성능에서 인기 평향 및 설명 가능성도 고려하여 책임감 있는 모델을 만들려고 한다.
- 딥러닝 하고 싶은데 비싸요. GPU 
- 그래서 선형 기반 추시랑 병행되고 있어요!




